In [5]:
import requests
import pandas as pd
import numpy as np
import sqlalchemy
import pytz
from datetime import datetime, timedelta
from pandas_market_calendars import get_calendar
from sklearn.linear_model import Lasso
from scipy.optimize import minimize

In [6]:
# Set random seed for reproducibility
np.random.seed(69)

engine = sqlalchemy.create_engine("sqlite:///C:/Users/lewis/OneDrive/tradingstrategies/databases/momentum_strategy_2_database.db")

In [7]:
stock_returns = pd.read_sql("SELECT * FROM stock_returns", con = engine)

In [63]:
def fit_lasso_for_date(date, net_returns, alpha=0.01):
    # Get returns up to the given date
    returns_up_to_date = net_returns.loc[:date]
    
    # Calculate expected returns
    expected_returns = returns_up_to_date.mean()
    
    # Fit the Lasso model
    lasso = Lasso(alpha=alpha, positive=True, max_iter=10000)
    lasso.fit(returns_up_to_date.values, expected_returns.values)
    
    # Get the Lasso weights
    lasso_weights = pd.Series(lasso.coef_, index=expected_returns.index)
    
    # Normalize weights to sum to 1
    lasso_weights = lasso_weights / lasso_weights.sum()
    
    return lasso_weights


In [8]:
stock_returns = stock_returns.set_index('index')
net_returns = stock_returns - 1

In [9]:
expected_returns = net_returns.mean()

In [10]:
cov_matrix = net_returns.cov()

In [24]:
#risk_free_rates = pd.read_sql("SELECT * FROM risk_free_rate", con=engine)

In [11]:
risk_free_rate = 0.02/252

In [65]:
def calculate_sharpe_ratio(returns, risk_free_rate=0):
    excess_returns = returns - risk_free_rate
    return np.mean(excess_returns) / np.std(excess_returns)

In [75]:
def fit_lasso_for_date_range(start_date, end_date, net_returns, alpha=0.01, risk_free_rate=0.02/252):
    # Get returns for the date range
    returns_in_range = net_returns.loc[start_date:end_date]
    
    # Calculate Sharpe ratio for an equally weighted portfolio as the target
    equal_weight_returns = returns_in_range.mean(axis=1)
    target_sharpe = calculate_sharpe_ratio(equal_weight_returns, risk_free_rate)
    
    # Fit the Lasso model
    lasso = Lasso(alpha=alpha, positive=True, fit_intercept=False, max_iter=10000)
    lasso.fit(returns_in_range, equal_weight_returns)
    
    # Get the Lasso weights
    lasso_weights = pd.Series(lasso.coef_, index=returns_in_range.columns)
    
    # Normalize weights to sum to 1
    lasso_weights = lasso_weights / lasso_weights.sum()
    
    return lasso_weights
    

In [78]:
# Example usage:
start_date = '2020-01-01'
end_date = '2020-12-31'
lasso_weights = fit_lasso_for_date_range(start_date, end_date, net_returns)
print(lasso_weights)

NVDA    0.0
TSLA    0.0
AAPL    0.0
AMD     0.0
MSFT    0.0
       ... 
MLI     0.0
TLRY    0.0
EXEL    0.0
UGI     0.0
HOLO    0.0
Length: 1000, dtype: float64
